In [1]:
import numpy as np
import random

# Магический квадрат

Размер магического квадрата

In [2]:
phrase = 'But if your life is such a big joke Why should I care?'

In [3]:
Size = 5

## Создание Магического квадрата

In [4]:
def createMagicSquare(size):
    N = size
    magic_square = np.zeros((N,N), dtype=int)

    n = 1
    i, j = 0, N//2

    while n <= N**2:
        magic_square[i, j] = n
        n += 1
        newi, newj = (i-1) % N, (j+1)% N
        if magic_square[newi, newj]:
            i += 1
        else:
            i, j = newi, newj
    
    return magic_square

In [5]:
magicSquare = createMagicSquare(Size)

In [6]:
magicSquare

array([[17, 24,  1,  8, 15],
       [23,  5,  7, 14, 16],
       [ 4,  6, 13, 20, 22],
       [10, 12, 19, 21,  3],
       [11, 18, 25,  2,  9]])

## Фраза

Поскольку длина нашей фразы больше размерности квадрата,
разобьем фразу на несколько квадратов

In [7]:
def chunkstring(string, length):
    return (string[0+i:length+i] for i in range(0, len(string), length))

In [8]:
phrasesBlocks = list(chunkstring(phrase, Size**2))

In [9]:
phrasesBlocks

['But if your life is such ', 'a big joke Why should I c', 'are?']

## Подстановка в квадрат

In [10]:
def turnInMagicSquare(square, phrase):
    sz = len(square)
    phraseSquare = np.empty((sz, sz), dtype='str')
    for row in range(sz):
        for col in range(sz):
            phraseSquare[row][col] = phrase[square[row][col] - 1]
    return phraseSquare

In [11]:
cryptSquares = []
for block in phrasesBlocks:
    while len(block) < Size ** 2:
        block += ' '
    cryptSquares.append(turnInMagicSquare(magicSquare, block))

In [12]:
cryptSquares

[array([[' ', 'h', 'B', 'y', 'f'],
        ['c', 'i', ' ', 'i', 'e'],
        [' ', 'f', 'l', ' ', 'u'],
        ['u', ' ', 's', 's', 't'],
        ['r', 'i', ' ', 'u', 'o']], dtype='<U1'),
 array([['h', ' ', 'a', 'o', ' '],
        ['I', 'g', 'j', 'y', 's'],
        ['i', ' ', 'h', 'l', ' '],
        ['e', 'W', 'u', 'd', 'b'],
        [' ', 'o', 'c', ' ', 'k']], dtype='<U1'),
 array([[' ', ' ', 'a', ' ', ' '],
        [' ', ' ', ' ', ' ', ' '],
        ['?', ' ', ' ', ' ', ' '],
        [' ', ' ', ' ', ' ', 'e'],
        [' ', ' ', ' ', 'r', ' ']], dtype='<U1')]

## Раскодировка

In [13]:
decryptedBlocks = ''
for crypted in cryptSquares:
    decrBlock = ''
    for decNum in range(Size**2):
        num = decNum + 1
        for i in range(Size):
            for j in range(Size):
                if magicSquare[i][j] == num:
                    decrBlock += crypted[i][j]
    decryptedBlocks += decrBlock

In [14]:
print(decryptedBlocks)

But if your life is such a big joke Why should I care?                     


# RSA

In [43]:
p = 827
q = 311
word = 'Анджеевский'

## Расширенный алгоритм Евклида

In [85]:
def bezout(a, b):
    '''An implementation of extended Euclidean algorithm.
    Returns integer x, y and gcd(a, b) for Bezout equation:
        ax + by = gcd(a, b).
    '''
    x, xx, y, yy = 1, 0, 0, 1
    while b:
        q = a // b
        a, b = b, a % b
        x, xx = xx, x - xx*q
        y, yy = yy, y - yy*q
    return (x, y, a)

In [86]:
def calcEuler(p, q):
    return (p - 1) * (q - 1)

In [87]:
n = p * q

In [88]:
def chooseE(n):
    euler = calcEuler(p , q)
    variants = [i for i in range(2, n - 1)]
    length = len(variants)
    while(length):
        index = random.randint(0, length)
        nod = bezout(euler, variants[index])[2]
        if nod == 1:
            return variants[index]
        else:
            del(variants[index])
            length -= 1

In [89]:
openKey = chooseE(n)

In [90]:
openKey

27053

## Проверка e

In [104]:
bezout(openKey, calcEuler(p, q))[2]

1

In [97]:
def chooseD(e):
    euler = calcEuler(p , q)
    bzt = bezout(euler, e)
    if bzt[1] >= 0:
        return bzt[1]
    else:
        return euler + bzt[1]

In [98]:
closeKey = chooseD(openKey)

In [99]:
closeKey

56857

## Проверка d 

In [101]:
(openKey * closeKey) % calcEuler(p , q)

1

## Шифрование

In [114]:
codes = [ord(i) for i in word]

In [115]:
codes

[1040, 1085, 1076, 1078, 1077, 1077, 1074, 1089, 1082, 1080, 1081]

In [116]:
crypted = [pow(i, openKey, n) for i in codes]

In [117]:
crypted

[96878,
 239442,
 137660,
 167646,
 29704,
 29704,
 156348,
 73152,
 72771,
 192024,
 245562]

# Дешифрование

In [119]:
decrypted = [pow(i, closeKey, n) for i in crypted]

In [120]:
decrypted

[1040, 1085, 1076, 1078, 1077, 1077, 1074, 1089, 1082, 1080, 1081]

In [121]:
result = ''
for i in decrypted:
    result += chr(i)

In [122]:
result

'Анджеевский'